# SVM
Training auf IDS18 80%  
Validierung auf IDS18 10%   
Test auf IDS18 10%  

In [ ]:
import os
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib

In [ ]:
# Maximale Zeilen und Spalten anzeigen
pd.set_option('display.max_rows', None)  # Zeilen
pd.set_option('display.max_columns', None)  # Spalten

# Logging Parameter
logging.basicConfig(
    #filename='',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Verzeichnisse der Datensätze
parquet_verzeichnis_ids17 = '../01_Datensaetze/improved_cic-ids-2017/ids17_parquet'
parquet_verzeichnis_ids18 = '../01_Datensaetze/improved_cse-cic-ids-2018/ids18_parquet'

# Hyperparameter Tuning durchführen
hyperparameter_tuning = False

### Laden von IDS18

In [ ]:
# ids18 Datensatz einlesen für Training 80%, Validierung 10% und Test 10%
ids18 = pd.read_parquet(os.path.join(parquet_verzeichnis_ids18 + '_prep_0'))
print("Class distribution\n{}".format(ids18.Label.value_counts()))

In [ ]:
print(ids18.shape)
print(ids18.columns)

### Trennen von Features und Labels

In [ ]:
X = ids18.iloc[:, :-1]  # Alle Spalten außer der letzten
print(f"Form von X: {X.shape}")
y = ids18.iloc[:, -1]   # Die letzte Spalte
print(f"Form von y: {y.shape}")

### Label Encoding für y

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"Einzigartige Labels: {label_encoder.classes_}")
print(f"Kodierte Labels: {np.unique(y_encoded)}")

### Skallierung von X

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Form von X: {X.shape}")
print(f"Form von X_scaled: {X_scaled.shape}")

### Aufteilen der Daten in Trainings- und Testdatensätze

In [ ]:
# Aufteilen in Trainings- und temporären Datensatz
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.20, random_state=42, stratify=y_encoded)

print(f"Form von X_train: {X_train.shape}")
print(f"Form von X_test: {X_test.shape}")

### Überprüfen der Klassenverteilung

In [ ]:
def print_class_distribution(y, dataset_name):
    unique, counts = np.unique(y, return_counts=True)
    total = len(y)
    print(f"Klassenverteilung in {dataset_name}:")
    for cls, count in zip(unique, counts):
        print(f"  Klasse {cls}: {count} Beispiele ({(count/total)*100:.2f}%)")
    print()

print_class_distribution(y_train, "Trainingsdatensatz")
print_class_distribution(y_test, "Testdatensatz")

### Optionales Hyperparameter-Tuning

In [ ]:
if hyperparameter_tuning:

    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    }

    grid_search = GridSearchCV(SVC(
        random_state=42), 
        param_grid, 
        cv=2, 
        scoring='accuracy', 
        n_jobs=24, 
        verbose=10)

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    logging.info(f"Beste Hyperparameter: {grid_search.best_params_}")

best_model = grid_search.best_estimator_
logging.info(f"Beste Hyperparameter: {grid_search.best_params_}")

### Training mit festen Parametern

In [ ]:
if not hyperparameter_tuning:
    best_model = SVC(C=1, gamma='auto', kernel='poly', random_state=42)
    best_model.fit(X_train, y_train)

### Evaluierung des Modells auf dem Testdatensatz

In [ ]:
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred)
logging.info(f"Test-Accuracy: {test_accuracy * 100:.2f}%")
logging.info("\nTest Classification Report:\n" + test_report)

### Klassifikationsbericht und eine Konfusionsmatrix

In [ ]:
# Klassifikationsbericht
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

# Konfusionsmatrix
cm = confusion_matrix(y_test, y_test_pred)

# Visualisierung der Konfusionsmatrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Vorhergesagte Klasse')
plt.ylabel('Wahre Klasse')
plt.title('Konfusionsmatrix')
plt.show()


### Speichern des Modells und der Vorverarbeitungsschritte

In [ ]:
joblib.dump(label_encoder, 'label_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(best_model, 'svm_model.pkl')